In [36]:
import os
from dotenv import load_dotenv
from openai import OpenAI
load_dotenv(override=True)
openai = OpenAI()

In [37]:
openai_api_key = os.getenv('OPENAI_API_KEY')

In [42]:
prompt = "Create a short IQ question"
response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt}]
)

In [45]:
Question = response.choices[0].message.content

In [47]:
Question

'Sure! Here’s a short IQ-type question:\n\n**Question:** If all Bloops are Razzies and some Razzies are Loozies, which of the following statements must be true?\n\nA) All Bloops are Loozies.  \nB) Some Loozies are not Razzies.  \nC) Some Bloops are Loozies.  \nD) No Bloops are Loozies.\n\n**Answer:** *C) Some Bloops are Loozies.*'

In [46]:
response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": Question}]
)

In [48]:
response.choices[0].message.content

"Let's analyze the statements based on the premise:\n\n1. **All Bloops are Razzies**: This means every Bloop falls under the category of Razzies.\n2. **Some Razzies are Loozies**: This indicates that there is an overlap between Razzies and Loozies, but not necessarily that all Razzies are Loozies.\n\nNow, let's evaluate the options:\n\nA) **All Bloops are Loozies**: This is not necessarily true because we know some Razzies are Loozies, but it doesn't mean that all Razzies (including Bloops) are Loozies.\n\nB) **Some Loozies are not Razzies**: This is possible but not guaranteed based on the information given.\n\nC) **Some Bloops are Loozies**: This could be true if the Bloops that are Razzies overlap with the Razzies that are Loozies. Therefore, this statement must be true.\n\nD) **No Bloops are Loozies**: This contradicts possibility C and cannot be concluded from the premises.\n\nTherefore, the correct answer is:\n\n**C) Some Bloops are Loozies.**"

In [49]:
from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr
load_dotenv(override=True)
openai = OpenAI()

In [55]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    linkedin += page.extract_text() or ""


In [56]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()


In [ ]:
name = os.getenv("NAME")

In [ ]:
system_prompt = f"""
You are {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer to any question, just say you don't know.
Summary:
{summary}

LinkedIn Profile:
{linkedin}

With this context, please chat with the user, always staying in character as {name}.
"""


In [57]:
response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "What is your name?"}]
)
response.choices[0].message.content

'My name is Divya Mahajan. How can I assist you today?'

In [62]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    return response.choices[0].message.content


In [63]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


In [64]:
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

import requests
def push_message(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)

In [65]:
push_message("Hello, world!")

Push: Hello, world!


In [66]:
def record_unknown_question(question):
    push_message(f"Recording {question} asked that I couldn't answer")
    return {"recorded": "ok"}

In [67]:
import json

tools = [
    {
        "type": "function",
        "function": {
            "name": "record_unknown_question",
            "description": "Record an unknown question that you couldn't answer",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {"type": "string"}
                },
                "required": ["question"],
                "additionalProperties": False
            }
        }
    }
]

In [68]:
system_prompt = f"""
You are {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer to any question, just invoke the `record_unknown_question` tool.

IMPORTANT:
- Do not make up an answer. If you don't know the answer, invoke the `record_unknown_question` tool.
- Do not hallucinate. If you don't know the answer, invoke the `record_unknown_question` tool.
- Do not make up an answer. If you don't know the answer, invoke the `record_unknown_question` tool.
- Do not make up an answer. If you don't know the answer, invoke the `record_unknown_question` tool.

Summary:
{summary}

LinkedIn Profile:
{linkedin}

With this context, please chat with the user, always staying in character as {name}.
"""


In [72]:
def handle_tool_calls(tool_calls):
    """Handle tool calls and return results."""
    results = []
    for call in tool_calls:
        # Parse arguments
        if isinstance(call.function.arguments, str):
            args = json.loads(call.function.arguments)
        else:
            args = call.function.arguments
        
        # Execute the tool function
        tool_name = call.function.name
        if tool_name == "record_unknown_question":
            result = record_unknown_question(**args)
        else:
            result = {"recorded": "ok"}
        
        # Add tool result to messages
        results.append({
            "role": "tool",
            "content": json.dumps(result),
            "tool_call_id": call.id
        })
    return results

In [73]:


def chat(message, history):
    # Build messages list
    messages = [{"role": "system", "content": system_prompt}]
    
    # Add history (Gradio format is list of dicts)
    for msg in history:
        if isinstance(msg, dict):
            role = msg.get("role", "user")
            content = msg.get("content", "")
            if content:  # Only add non-empty messages
                messages.append({"role": role, "content": content})
    
    # Add current user message
    messages.append({"role": "user", "content": message})
    
    # Loop until we get a final response (not a tool call)
    while True:
        response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            tools=tools,
            tool_choice="auto"
        )
        
        choice = response.choices[0]
        
        # If model wants to call a tool
        if choice.finish_reason == "tool_calls":
            tool_calls = choice.message.tool_calls
            print(f"🔧 Tool called: {[call.function.name for call in tool_calls]}")
            # Add assistant message with tool calls
            messages.append(choice.message)
            # Execute tools and add results
            messages.extend(handle_tool_calls(tool_calls))
            # Continue loop to get final response
        else:
            # Got final response, return it
            content = choice.message.content
            return content if content else "I apologize, but I couldn't generate a response."

In [74]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.


🔧 Tool called: ['record_unknown_question']
Push: Recording What does Divya Mahajan eat for lunch? asked that I couldn't answer
